In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

a4_dims = (11.7, 8.27)
sns.set(font_scale = 2)

In [3]:
fa_17 = pd.read_csv('UD75_data\Fa17UD75_hashed.csv')
fa_18 = pd.read_csv('UD75_data\Fa18UD75_hashed.csv')
fa_19 = pd.read_csv('UD75_data\Fa19UD75_hashed.csv')
sp_18 = pd.read_csv('UD75_data\Sp18UD75_hashed.csv')
sp_19 = pd.read_csv('UD75_data\Sp19UD75_hashed.csv')

fa_19_allls = pd.read_csv('UD75_data\Fa19AllLS_hashed.csv')
fa_19_nomajor = pd.read_csv('UD75_data\Fa19NoMajor_hashed.csv')

ad_fa_19_ycbm = pd.read_csv('UD75_data\SuFa19YCBM_hashed.csv')
ad_fa_19_di = pd.read_csv('UD75_data\Fa19DI_hashed.csv')

fa_17_v2 = pd.read_csv("UD75 v2\Fa17UD75_v2_hashed.csv")
sp_18_v2 = pd.read_csv("UD75 v2\Sp18UD75_v2_hashed.csv")
fa_18_v2 = pd.read_csv("UD75 v2\Fa18UD75_v2_hashed.csv")
sp_19_v2 = pd.read_csv("UD75 v2\Sp19UD75_v2_hashed.csv")
fa_19_v2 = pd.read_csv("UD75 v2\Fa19UD75_v2_hashed.csv")

In [26]:
fa_17.shape

(2411, 10)

In [27]:
fa_17_v2.shape

(4801, 20)

In [28]:
fa_17_v2.columns

Index(['Unnamed: 0', 'SID', 'Include Sb Week Desc', 'Withdrawal Reason Eff Dt',
       'Entry Status Cd', 'Educ Non Exam Level Cd',
       'Academic Department Short Nm - Major', 'Cum Gpa No',
       'Cum Total Units Less Exam No', 'Cum Transfer Units No',
       'Cum Total Letgrd Attempted No', 'Cum Ucb Letgrd Units No',
       'Cum Pnp Units No', 'Cum Total Incomplete No',
       'Term Letgrd Units Attempted No', 'Eot Term Letgrd Units Complete',
       'Eot Trm Tot Letgrd Grd Pnts No', 'Eot Term Pnp Units Completed',
       'Term Total Units Attempted No', 'Eot Term Total Units Completed'],
      dtype='object')

In [29]:
fa_17.columns

Index(['Unnamed: 0', 'Student Id', 'Entry Status Cd', 'Educ Non Exam Level Cd',
       'Academic Department Short Nm - Major', 'Cum Gpa No',
       'Cum Ucb Letgrd Units No', 'Eot Term Letgrd Units Complete',
       'Eot Term Pnp Units Completed', 'Eot Trm Tot Letgrd Grd Pnts No'],
      dtype='object')

In [30]:
fa_17_v2[fa_17_v2['Include Sb Week Desc'] == 'EOT'].shape

(2411, 20)

In [31]:
set(fa_17.columns).difference(set(fa_17_v2.columns))

{'Student Id'}

In [32]:
set(fa_17_v2.columns).difference(set(fa_17.columns))

{'Cum Pnp Units No',
 'Cum Total Incomplete No',
 'Cum Total Letgrd Attempted No',
 'Cum Total Units Less Exam No',
 'Cum Transfer Units No',
 'Eot Term Total Units Completed',
 'Include Sb Week Desc',
 'SID',
 'Term Letgrd Units Attempted No',
 'Term Total Units Attempted No',
 'Withdrawal Reason Eff Dt'}

In [47]:
all_data_v2 = [fa_17_v2, sp_18_v2, fa_18_v2, sp_19_v2, fa_19_v2]
all_student_df = pd.DataFrame({'SID': []})
i = 1
year = 17

for df in all_data_v2:
    term = '_fa' if i%2 != 0 else '_sp'
    year = year + 1 if i%2 == 0 else year
    eot_string = term + '_' + str(year) + '_eot'
    cen_string = term + '_' + str(year) + '_cen'
    df_eot = df[df['Include Sb Week Desc'] == 'EOT']
    df_cen = df[(df['Include Sb Week Desc'] == 'CEN') & (df['SID'].isin(df_eot['SID']))]
    df_eot = df_eot.drop(['Unnamed: 0', 'Include Sb Week Desc'], axis = 1)
    df_cen = df_cen.drop(['Unnamed: 0', 'Include Sb Week Desc'], axis = 1)
    clean_df = pd.merge(df_cen, df_eot, how = 'inner', on = 'SID', sort = True, suffixes = (cen_string, eot_string))
    all_student_df = pd.merge(all_student_df, clean_df, how = 'outer', on = 'SID', sort = True)
    i+=1

all_student_df

,SID,Withdrawal Reason Eff Dt_fa_17_cen,Entry Status Cd_fa_17_cen,Educ Non Exam Level Cd_fa_17_cen,Academic Department Short Nm - Major_fa_17_cen,Cum Gpa No_fa_17_cen,Cum Total Units Less Exam No_fa_17_cen,Cum Transfer Units No_fa_17_cen,Cum Total Letgrd Attempted No_fa_17_cen,Cum Ucb Letgrd Units No_fa_17_cen,...,Eot Trm Tot Letgrd Grd Pnts No_fa_19_eot,Cum Pnp Units No_fa_19_eot,Cum Total Incomplete No_fa_19_eot,Cum Total Units Less Exam No_fa_19_eot,Cum Transfer Units No_fa_19_eot,Cum Total Letgrd Attempted No_fa_19_eot,Eot Term Total Units Completed_fa_19_eot,Term Letgrd Units Attempted No_fa_19_eot,Term Total Units Attempted No_fa_19_eot,Withdrawal Reason Eff Dt_fa_19_eot
0,00050252ff312c7371e9f4a0809c1d60ecdcc969dd89f0...,NaN,NF,3,L&S Undeclared,3.594,81.0,33.0,37.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0015c649711f865c2247411166d9980a3c89e00dae15a6...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00289053f169dff564e196328b98245010498f52498880...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.0,5.0,0.0,107.0,49.0,63.0,13.0,12.0,13.0,NaN
3,0029df9808bd845e4adf3c6dee29432743ed62e6a4afbc...,NaN,NF,3,L&S Undeclared,2.925,65.0,0.0,63.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00358163d10366a087e31307d26b09373ef9b6aac8303e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6063,ffc4f900b936e11ecd8b4e8760a7aecb945af77d5f36dd...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6064,ffc96951ffc345d8fb7b115401b4d0884ff111a64bd8b0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6065,ffced42d0fddf511561af9ffa16b52e7ce1095f504c788...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6066,ffdc130809d8b6f58067a9e4cf3f9bcea8d7bac75a78b4...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
holds = pd.DataFrame({'SID' : all_student_df['SID'], 'Holds' : [[] for _ in range(all_student_df.shape[0])]})
holds

,SID,Holds
0,00050252ff312c7371e9f4a0809c1d60ecdcc969dd89f0...,[]
1,0015c649711f865c2247411166d9980a3c89e00dae15a6...,[]
2,00289053f169dff564e196328b98245010498f52498880...,[]
3,0029df9808bd845e4adf3c6dee29432743ed62e6a4afbc...,[]
4,00358163d10366a087e31307d26b09373ef9b6aac8303e...,[]
...,...,...
6063,ffc4f900b936e11ecd8b4e8760a7aecb945af77d5f36dd...,[]
6064,ffc96951ffc345d8fb7b115401b4d0884ff111a64bd8b0...,[]
6065,ffced42d0fddf511561af9ffa16b52e7ce1095f504c788...,[]
6066,ffdc130809d8b6f58067a9e4cf3f9bcea8d7bac75a78b4...,[]


In [81]:
holds = pd.DataFrame({'SID' : all_student_df['SID'], 'Holds' : [[] for _ in range(all_student_df.shape[0])]})

for index in holds.index:
    i = 1
    h = []
    for df in all_data_v2:
        
        if (holds.loc[index, 'SID'] in list(df['SID'].astype(str))):
            h = h + [i]
            
        i += 1
    holds.loc[index, 'Holds'] = h
holds
# all_student_df['Holds']            

,SID,Holds
0,00050252ff312c7371e9f4a0809c1d60ecdcc969dd89f0...,[1]
1,0015c649711f865c2247411166d9980a3c89e00dae15a6...,[1]
2,00289053f169dff564e196328b98245010498f52498880...,"[4, 5]"
3,0029df9808bd845e4adf3c6dee29432743ed62e6a4afbc...,"[1, 2, 3]"
4,00358163d10366a087e31307d26b09373ef9b6aac8303e...,[2]
...,...,...
6063,ffc4f900b936e11ecd8b4e8760a7aecb945af77d5f36dd...,[1]
6064,ffc96951ffc345d8fb7b115401b4d0884ff111a64bd8b0...,[3]
6065,ffced42d0fddf511561af9ffa16b52e7ce1095f504c788...,"[1, 2]"
6066,ffdc130809d8b6f58067a9e4cf3f9bcea8d7bac75a78b4...,"[1, 2, 3]"


In [82]:
all_student_df['Holds'] = holds['Holds']
all_student_df

,SID,Withdrawal Reason Eff Dt_fa_17_cen,Entry Status Cd_fa_17_cen,Educ Non Exam Level Cd_fa_17_cen,Academic Department Short Nm - Major_fa_17_cen,Cum Gpa No_fa_17_cen,Cum Total Units Less Exam No_fa_17_cen,Cum Transfer Units No_fa_17_cen,Cum Total Letgrd Attempted No_fa_17_cen,Cum Ucb Letgrd Units No_fa_17_cen,...,Cum Pnp Units No_fa_19_eot,Cum Total Incomplete No_fa_19_eot,Cum Total Units Less Exam No_fa_19_eot,Cum Transfer Units No_fa_19_eot,Cum Total Letgrd Attempted No_fa_19_eot,Eot Term Total Units Completed_fa_19_eot,Term Letgrd Units Attempted No_fa_19_eot,Term Total Units Attempted No_fa_19_eot,Withdrawal Reason Eff Dt_fa_19_eot,Holds
0,00050252ff312c7371e9f4a0809c1d60ecdcc969dd89f0...,NaN,NF,3,L&S Undeclared,3.594,81.0,33.0,37.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1]
1,0015c649711f865c2247411166d9980a3c89e00dae15a6...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1]
2,00289053f169dff564e196328b98245010498f52498880...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,0.0,107.0,49.0,63.0,13.0,12.0,13.0,NaN,"[4, 5]"
3,0029df9808bd845e4adf3c6dee29432743ed62e6a4afbc...,NaN,NF,3,L&S Undeclared,2.925,65.0,0.0,63.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1, 2, 3]"
4,00358163d10366a087e31307d26b09373ef9b6aac8303e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6063,ffc4f900b936e11ecd8b4e8760a7aecb945af77d5f36dd...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1]
6064,ffc96951ffc345d8fb7b115401b4d0884ff111a64bd8b0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[3]
6065,ffced42d0fddf511561af9ffa16b52e7ce1095f504c788...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1, 2]"
6066,ffdc130809d8b6f58067a9e4cf3f9bcea8d7bac75a78b4...,NaN,AD,3,L&S Undeclared,0.000,70.0,70.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1, 2, 3]"


Stuff to do after this: make column for Declaration(True or False), make column for withdrawals(which semesters they withdrew).